### 主表日期修改

In [1]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel("./data/raw/股份回购.xlsx", sheet_name='股份回购')

# 提取BPAmtDy列的年份
def extract_year(date_value):
    """提取日期年份的辅助函数"""
    try:
        if pd.isna(date_value):
            return None
        # 如果是字符串格式
        if isinstance(date_value, str):
            # 尝试按'-'分割提取年份
            if '-' in date_value:
                parts = date_value.split('-')
                return int(parts[0]) if parts[0].isdigit() else None
            # 或者直接提取前4位
            elif len(date_value) >= 4 and date_value[:4].isdigit():
                return int(date_value[:4])
        # 如果是日期时间类型
        elif isinstance(date_value, (pd.Timestamp, datetime)):
            return date_value.year
    except Exception:
        return None
    return None

# 将BPAmtDy列转换为年份
df['Year'] = df['BPAmtDy'].apply(extract_year)

# 保存处理后的数据到新文件
df.to_excel('./data/interim/1_股份回购_仅改年份.xlsx', index=False)

print("数据处理完成！")
print(f"处理后的数据行数: {len(df)}")
print("已将日期改为年份，未删除任何记录")
print(f"已保存到: 股份回购_仅改年份.xlsx")

# 显示前几行数据检查
print("\n处理后数据示例：")
print(df[['SCode', 'BPAmtDy']].head(10))

数据处理完成！
处理后的数据行数: 11465
已将日期改为年份，未删除任何记录
已保存到: 股份回购_仅改年份.xlsx

处理后数据示例：
    SCode     BPAmtDy
0    股票代码    董事会预案公告日
1  000002  2015-07-06
2  000002  2022-03-31
3  000004  2021-05-27
4  000004  2022-04-30
5  000004  2023-04-29
6  000005  2018-11-29
7  000008  2017-05-10
8  000008  2017-08-30
9  000008  2017-12-15


### 副表合并

In [2]:
import pandas as pd

# 读取两个Excel文件
df_repo = pd.read_excel('./data/interim/1_股份回购_仅改年份.xlsx')
df_div = pd.read_excel('./data/raw/分红实施内容.xlsx')

print("原始数据统计：")
print(f"股份回购表: {len(df_repo)} 行，{len(df_repo.columns)} 列")
print(f"分红实施表: {len(df_div)} 行，{len(df_div.columns)} 列")
print("-" * 50)

# 显示两个表的列名
print("股份回购表列名：")
print(df_repo.columns.tolist())
print("\n分红实施表列名：")
print(df_div.columns.tolist())
print("-" * 50)

# 处理股票代码函数
def clean_stock_code(code):
    """清理股票代码，去掉后缀"""
    if pd.isna(code):
        return None
    code_str = str(code).strip()
    # 处理各种可能的后缀格式
    if '.' in code_str:
        parts = code_str.split('.')
        return parts[0]
    # 如果是数字，转为6位字符串
    if code_str.isdigit():
        return code_str.zfill(6)
    return code_str

# 处理两个表的股票代码
df_repo['SCode_clean'] = df_repo['SCode'].apply(clean_stock_code)
df_div['SCode_clean'] = df_div['SCode'].apply(clean_stock_code)

# 确保年份列类型一致
# 重命名股份回购表的年份列

# 将年份列转换为整数类型（如果有空值，则转换为浮点型）
df_repo['Year'] = pd.to_numeric(df_repo['Year'], errors='coerce')
df_div['Year'] = pd.to_numeric(df_div['Year'], errors='coerce')

# 合并前的数据预览
print("股份回购表年份范围:", df_repo['Year'].min(), "-", df_repo['Year'].max())
print("分红实施表年份范围:", df_div['Year'].min(), "-", df_div['Year'].max())
print("-" * 50)

# 执行合并
merged_df = pd.merge(df_repo, 
                     df_div, 
                     left_on=['SCode_clean', 'Year'], 
                     right_on=['SCode_clean', 'Year'], 
                     how='left',
                     suffixes=('_回购', '_分红'))

# 删除临时列
merged_df = merged_df.drop(columns=['SCode_clean'])

# 保存合并结果
output_file = './data/interim/2_股份回购与分红合并.xlsx'
merged_df.to_excel(output_file, index=False)

print("数据合并完成！")
print("-" * 50)
print("合并结果统计：")
print(f"合并后总行数: {len(merged_df)}")
print(f"成功匹配的行数: {merged_df['CoName'].notna().sum()}")
print(f"未匹配的行数: {merged_df['CoName'].isna().sum()}")
print(f"已保存到: {output_file}")

# 显示匹配和未匹配的样本
print("\n成功匹配的样本（前3行）：")
matched_samples = merged_df[merged_df['CoName'].notna()].head(3)
print(matched_samples[['SCode_回购', 'Year', 'CoName']])

print("\n未匹配的样本（前3行）：")
unmatched_samples = merged_df[merged_df['CoName'].isna()].head(3)
print(unmatched_samples[['SCode_回购', 'Year']])

原始数据统计：
股份回购表: 11465 行，15 列
分红实施表: 40605 行，15 列
--------------------------------------------------
股份回购表列名：
['SCode', 'BPAmtDy', 'SoAmDAD', 'RepMaNum', 'RepMiNum', 'RepType', 'RepProc', 'MaxPrc', 'MinPrc', 'AvePrc', 'RepSN', 'RepAmt', 'RepSDat', 'RepDDat', 'Year']

分红实施表列名：
['SCode', 'CoName', 'Year', 'ImProgress', 'IDPR_PreTax', 'IDPR_AftTax', 'IBnsIssRt', 'IEqTnRt', 'EqBenmk', 'EqBenmkDt', 'AnnDt', 'RecordDt', 'DRDt', 'DPRDt', 'BnsShrLDt']
--------------------------------------------------
股份回购表年份范围: 2005.0 - 2024.0
分红实施表年份范围: 1991.0 - 2024.0
--------------------------------------------------
数据合并完成！
--------------------------------------------------
合并结果统计：
合并后总行数: 11465
成功匹配的行数: 6543
未匹配的行数: 4922
已保存到: ./data/interim/2_股份回购与分红合并.xlsx

成功匹配的样本（前3行）：
  SCode_回购    Year CoName
0     股票代码     NaN   公司简称
1   000002  2015.0    万科A
7   000008  2017.0   神州高铁

未匹配的样本（前3行）：
  SCode_回购    Year
2   000002  2022.0
3   000004  2021.0
4   000004  2022.0


### 得到Treat值

In [3]:
import pandas as pd
import numpy as np

# 读取合并后的Excel文件
df = pd.read_excel('./data/interim/2_股份回购与分红合并.xlsx')

# 检查列名，确认是否有SCode_分红列
print("数据框的列名：")
print(df.columns.tolist())

# 查看SCode_分红列的情况
if 'SCode_分红' in df.columns:
    print(f"\nSCode_分红列的情况：")
    print(f"非空值数量：{df['SCode_分红'].notna().sum()}")
    print(f"空值数量：{df['SCode_分红'].isna().sum()}")
    
    # 创建Treat列
    df['Treat'] = df['SCode_分红'].apply(lambda x: 1 if pd.notna(x) else 0)
    
    # 或者更简洁的方式：
    # df['Treat'] = df['SCode_分红'].notna().astype(int)
    
else:
    print("未找到SCode_分红列，尝试查找其他可能的列名...")
    # 查找包含"分红"的列名
    div_cols = [col for col in df.columns if '分红' in str(col)]
    print(f"找到的包含'分红'的列：{div_cols}")
    
    if div_cols:
        # 使用第一个包含"分红"的列
        div_col = div_cols[0]
        print(f"使用列 '{div_col}' 来创建Treat列")
        df['Treat'] = df[div_col].apply(lambda x: 1 if pd.notna(x) else 0)
    else:
        # 如果找不到相关列，尝试找公司名称列
        if 'CoName' in df.columns:
            print("使用CoName列来创建Treat列")
            df['Treat'] = df['CoName'].apply(lambda x: 1 if pd.notna(x) else 0)
        else:
            # 查找是否有任何包含公司名称的列
            name_cols = [col for col in df.columns if 'Name' in str(col) or '公司' in str(col)]
            if name_cols:
                print(f"使用列 '{name_cols[0]}' 来创建Treat列")
                df['Treat'] = df[name_cols[0]].apply(lambda x: 1 if pd.notna(x) else 0)
            else:
                print("未找到合适的列来创建Treat列")

# 显示Treat列的统计信息
if 'Treat' in df.columns:
    print(f"\nTreat列统计：")
    print(f"Treat=1的行数：{df['Treat'].sum()}")
    print(f"Treat=0的行数：{(df['Treat']==0).sum()}")
    print(f"Treat=1的比例：{df['Treat'].mean():.2%}")
    


# 保存处理后的文件
output_file = './data/interim/3_股份回购与分红合并股份回购与分红合并_添加Treat.xlsx'
df.to_excel(output_file, index=False)
print(f"\n文件已保存为：{output_file}")

数据框的列名：
['SCode_回购', 'BPAmtDy', 'SoAmDAD', 'RepMaNum', 'RepMiNum', 'RepType', 'RepProc', 'MaxPrc', 'MinPrc', 'AvePrc', 'RepSN', 'RepAmt', 'RepSDat', 'RepDDat', 'Year', 'SCode_分红', 'CoName', 'ImProgress', 'IDPR_PreTax', 'IDPR_AftTax', 'IBnsIssRt', 'IEqTnRt', 'EqBenmk', 'EqBenmkDt', 'AnnDt', 'RecordDt', 'DRDt', 'DPRDt', 'BnsShrLDt']

SCode_分红列的情况：
非空值数量：6543
空值数量：4922

Treat列统计：
Treat=1的行数：6543
Treat=0的行数：4922
Treat=1的比例：57.07%

文件已保存为：./data/interim/3_股份回购与分红合并股份回购与分红合并_添加Treat.xlsx


### 得到Post

In [4]:
import pandas as pd
import numpy as np

# 读取已添加Treat列的文件
df = pd.read_excel('./data/interim/3_股份回购与分红合并股份回购与分红合并_添加Treat.xlsx')

print("数据基本信息：")
print(f"总行数：{len(df)}")
print(f"总列数：{len(df.columns)}")
print(f"列名：{df.columns.tolist()}")
print("-" * 50)

# 检查年份列的名称
# 可能是'Year_repo'、'Year'或其他名称
year_columns = [col for col in df.columns if 'Year' in str(col) or 'year' in str(col)]
print(f"找到的年份相关列：{year_columns}")

if year_columns:
    # 通常使用第一个年份列
    year_col = year_columns[0]
    print(f"使用列 '{year_col}' 来判断年份")
    
    # 确保年份列是数值类型
    df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
    
    # 创建Post列：2023年及以前为0，之后为1
    df['Post'] = df[year_col].apply(lambda x: 0 if pd.notna(x) and x <= 2023 else 1 if pd.notna(x) else np.nan)
    
    # 统计Post列的值
    print("\nPost列统计：")
    print(f"Post=0 (≤2023年) 的行数：{(df['Post']==0).sum()}")
    print(f"Post=1 (>2023年) 的行数：{(df['Post']==1).sum()}")
    print(f"Post为空的行程：{df['Post'].isna().sum()}")
    
    # 显示年份范围
    print(f"\n年份范围：{df[year_col].min()} - {df[year_col].max()}")
    
    # 按年份和Post分组统计
    print("\n按年份和Post的分布：")
    year_post_count = df.groupby([year_col, 'Post']).size().unstack(fill_value=0)
    print(year_post_count.head(10))
    
else:
    print("未找到年份列！")
    print("可用的列名：")
    for col in df.columns:
        print(f"  {col}: 数据类型={df[col].dtype}, 示例={df[col].iloc[0] if len(df) > 0 else '空'}")
    
    # 尝试手动指定年份列
    # 这里假设可能叫'BPAmtDy'或其他名称
    possible_year_cols = ['BPAmtDy', 'Year', 'year', '年度', '会计年度']
    for col in possible_year_cols:
        if col in df.columns:
            year_col = col
            print(f"\n手动找到年份列: {year_col}")
            break
    else:
        # 如果没有找到，让用户输入
        year_col = input("请输入年份列的名称: ")
    
    # 确保年份列是数值类型
    df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
    
    # 创建Post列
    df['Post'] = df[year_col].apply(lambda x: 0 if pd.notna(x) and x <= 2023 else 1 if pd.notna(x) else np.nan)

# 保存修改后的文件
output_file = './data/processed/股份回购与分红合并_添加Treat_Post.xlsx'
df.to_excel(output_file, index=False)
print(f"\n文件已保存为：{output_file}")

# 显示添加后的列
print(f"\n添加Post列后的列名：")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2}. {col}")

# 保存一份简明的统计报告
with open('Post列添加统计.txt', 'w', encoding='utf-8') as f:
    f.write("Post列添加统计报告\n")
    f.write("=" * 50 + "\n")
    f.write(f"总行数: {len(df)}\n")
    f.write(f"Post=0 (≤2023年): {(df['Post']==0).sum()} ({df['Post'].eq(0).mean():.2%})\n")
    f.write(f"Post=1 (>2023年): {(df['Post']==1).sum()} ({df['Post'].eq(1).mean():.2%})\n")
    f.write(f"Post为空值: {df['Post'].isna().sum()} ({df['Post'].isna().mean():.2%})\n")
    
    if year_columns:
        year_col = year_columns[0]
        # 按Post分组显示年份范围
        post_0_years = df[df['Post']==0][year_col].unique()
        post_1_years = df[df['Post']==1][year_col].unique()
        
        f.write(f"\nPost=0的年份: {sorted(post_0_years) if len(post_0_years) > 0 else '无'}\n")
        f.write(f"Post=1的年份: {sorted(post_1_years) if len(post_1_years) > 0 else '无'}\n")

print(f"\n统计报告已保存到：Post列添加统计.txt")

数据基本信息：
总行数：11465
总列数：30
列名：['SCode_回购', 'BPAmtDy', 'SoAmDAD', 'RepMaNum', 'RepMiNum', 'RepType', 'RepProc', 'MaxPrc', 'MinPrc', 'AvePrc', 'RepSN', 'RepAmt', 'RepSDat', 'RepDDat', 'Year', 'SCode_分红', 'CoName', 'ImProgress', 'IDPR_PreTax', 'IDPR_AftTax', 'IBnsIssRt', 'IEqTnRt', 'EqBenmk', 'EqBenmkDt', 'AnnDt', 'RecordDt', 'DRDt', 'DPRDt', 'BnsShrLDt', 'Treat']
--------------------------------------------------
找到的年份相关列：['Year']
使用列 'Year' 来判断年份

Post列统计：
Post=0 (≤2023年) 的行数：9729
Post=1 (>2023年) 的行数：1735
Post为空的行程：1

年份范围：2005.0 - 2024.0

按年份和Post的分布：
Post    0.0  1.0
Year            
2005.0    4    0
2006.0    2    0
2008.0    2    0
2009.0    1    0
2010.0    2    0
2011.0   10    0
2012.0   48    0
2013.0  110    0
2014.0  199    0
2015.0  300    0

文件已保存为：./data/processed/股份回购与分红合并_添加Treat_Post.xlsx

添加Post列后的列名：
 1. SCode_回购
 2. BPAmtDy
 3. SoAmDAD
 4. RepMaNum
 5. RepMiNum
 6. RepType
 7. RepProc
 8. MaxPrc
 9. MinPrc
10. AvePrc
11. RepSN
12. RepAmt
13. RepSDat
14. RepDDat
15. Year